In [21]:
# collision_sim_3d.py
# 3D simulation with collision detection highlighting and interactive camera.
# Methods: Brute Force, Sweep-and-Prune (1D on X), BVH with refit.
#
# Save as collision_sim_3d.py and run with: python collision_sim_3d.py
# Requires pygame.

import time, random, math, sys
from collections import namedtuple

try:
    import pygame
    PYGAME_AVAILABLE = True
except Exception:
    PYGAME_AVAILABLE = False

Vec3 = namedtuple("Vec3", ["x","y","z"])
AABB = namedtuple("AABB", ["minx","miny","minz","maxx","maxy","maxz"])

def make_aabb_from_center_radius(cx,cy,cz,r):
    return AABB(cx-r, cy-r, cz-r, cx+r, cy+r, cz+r)

def aabb_intersect(a: AABB, b: AABB):
    return (a.minx <= b.maxx and a.maxx >= b.minx and
            a.miny <= b.maxy and a.maxy >= b.miny and
            a.minz <= b.maxz and a.maxz >= b.minz)

# ---------------- Brute Force ----------------
def brute_force_collisions(aabbs):
    n = len(aabbs)
    collisions = []
    for i in range(n):
        ai = aabbs[i]
        for j in range(i+1, n):
            if aabb_intersect(ai, aabbs[j]):
                collisions.append((i,j))
    return collisions

# ---------------- Sweep and Prune (1D on X) ----------------
def sweep_and_prune_collisions(aabbs):
    n = len(aabbs)
    entries = [(aabbs[i].minx, aabbs[i].maxx, i) for i in range(n)]
    entries.sort(key=lambda e: e[0])
    active = []
    collisions = []
    for minx, maxx, idx in entries:
        # remove inactive
        new_active = []
        for (a_maxx, j) in active:
            if a_maxx >= minx:
                new_active.append((a_maxx, j))
        active = new_active
        for a_maxx, j in active:
            if aabb_intersect(aabbs[idx], aabbs[j]):
                collisions.append((idx, j))
        active.append((maxx, idx))
    return collisions

# ---------------- Morton / BVH utilities ----------------
def expand_bits_10(v):
    out = 0
    for i in range(10):
        bit = (v >> i) & 1
        out |= bit << (3*i)
    return out

def morton3_from_point(x,y,z, world_size=100.0):
    def norm(v):
        return min(max((v + world_size/2.0) / world_size, 0.0), 1.0)
    xi = int(norm(x) * 1023.0) & 0x3FF
    yi = int(norm(y) * 1023.0) & 0x3FF
    zi = int(norm(z) * 1023.0) & 0x3FF
    xx = expand_bits_10(xi)
    yy = expand_bits_10(yi)
    zz = expand_bits_10(zi)
    return xx | (yy << 1) | (zz << 2)

class BVHNode:
    __slots__ = ("left","right","box_id","aabb")
    def __init__(self):
        self.left = None
        self.right = None
        self.box_id = -1
        self.aabb = AABB(float('inf'),float('inf'),float('inf'),
                         -float('inf'),-float('inf'),-float('inf'))
    def is_leaf(self):
        return self.box_id != -1

def combine_aabb(a,b):
    return AABB(min(a.minx,b.minx), min(a.miny,b.miny), min(a.minz,b.minz),
                max(a.maxx,b.maxx), max(a.maxy,b.maxy), max(a.maxz,b.maxz))

def build_bvh_with_leaf_map(centers, aabbs):
    lst = []
    world_guess = max(1.0, max(abs(c[0]) for c in centers+[ (0,0,0) ])*2)
    for i, c in enumerate(centers):
        m = morton3_from_point(c[0], c[1], c[2], world_size=world_guess)
        lst.append({'id': i, 'morton': m})
    lst.sort(key=lambda e: e['morton'])
    if not lst:
        return None, {}

    def create_subtree_with_map(sorted_list, begin, end, aabbs, leaf_map):
        if begin == end:
            node = BVHNode()
            node.box_id = sorted_list[begin]['id']
            node.aabb = aabbs[node.box_id]
            leaf_map[node.box_id] = node
            return node
        m = (begin + end) // 2
        node = BVHNode()
        node.left = create_subtree_with_map(sorted_list, begin, m, aabbs, leaf_map)
        node.right = create_subtree_with_map(sorted_list, m+1, end, aabbs, leaf_map)
        node.aabb = combine_aabb(node.left.aabb, node.right.aabb)
        return node

    leaf_map = {}
    root = create_subtree_with_map(lst, 0, len(lst)-1, aabbs, leaf_map)
    return root, leaf_map

def refit_bvh(node):
    if node is None:
        return None
    if node.is_leaf():
        return node.aabb
    left_aabb = refit_bvh(node.left)
    right_aabb = refit_bvh(node.right)
    node.aabb = combine_aabb(left_aabb, right_aabb)
    return node.aabb

def find_collisions_for(box_id, aabb, node, aabbs, collisions):
    if node is None:
        return
    if not aabb_intersect(aabb, node.aabb):
        return
    if node.is_leaf():
        if node.box_id != box_id:
            if aabb_intersect(aabb, aabbs[node.box_id]):
                collisions.append((box_id, node.box_id))
        return
    find_collisions_for(box_id, aabb, node.left, aabbs, collisions)
    find_collisions_for(box_id, aabb, node.right, aabbs, collisions)

def bvh_collisions_using_root(aabbs, root):
    collisions = []
    for i,a in enumerate(aabbs):
        find_collisions_for(i, a, root, aabbs, collisions)
    return collisions

# ---------------- Camera (orbit) & projection ----------------
class Camera:
    def __init__(self, yaw=0.0, pitch=0.0, distance=600.0, target=(0.0,0.0,0.0)):
        self.yaw = yaw    # rotation around Y (left-right)
        self.pitch = pitch  # rotation around X (up-down)
        self.distance = distance
        self.target = list(target)  # center we orbit around
        self.min_distance = 50.0
        self.max_distance = 2000.0

    def get_position(self):
        # spherical -> cartesian
        cy = math.cos(self.yaw); sy = math.sin(self.yaw)
        cp = math.cos(self.pitch); sp = math.sin(self.pitch)
        x = self.distance * cp * sy + self.target[0]
        y = self.distance * sp + self.target[1]
        z = self.distance * cp * cy + self.target[2]
        return [x,y,z]

    def world_to_camera(self, p):
        # Transform a world point into camera space (camera looks to target)
        cam_pos = self.get_position()
        # build camera basis: forward (target - cam), right, up
        fx = self.target[0] - cam_pos[0]
        fy = self.target[1] - cam_pos[1]
        fz = self.target[2] - cam_pos[2]
        # normalize forward
        flen = math.sqrt(fx*fx + fy*fy + fz*fz) or 1e-6
        fx/=flen; fy/=flen; fz/=flen
        # arbitrary world up
        upx, upy, upz = 0.0, 1.0, 0.0
        # right = forward x up
        rx = fy*upz - fz*upy
        ry = fz*upx - fx*upz
        rz = fx*upy - fy*upx
        rlen = math.sqrt(rx*rx+ry*ry+rz*rz) or 1e-6
        rx/=rlen; ry/=rlen; rz/=rlen
        # true up = right x forward
        ux = ry*fz - rz*fy
        uy = rz*fx - rx*fz
        uz = rx*fy - ry*fx
        # translate point relative to cam
        vx = p[0] - cam_pos[0]
        vy = p[1] - cam_pos[1]
        vz = p[2] - cam_pos[2]
        # coordinates in camera space
        cx = vx*rx + vy*ry + vz*rz
        cy = vx*ux + vy*uy + vz*uz
        cz = vx*fx + vy*fy + vz*fz  # forward axis
        return (cx, cy, cz)

def project_point(p, cam: Camera, SW, SH, fov=60.0):
    # p: world point (x,y,z)
    cx, cy, cz = cam.world_to_camera(p)
    # perspective projection: map camera forward axis (cz) to screen
    # if cz <= 0 (behind camera), we still project but can mark as behind
    if cz <= 1e-4:
        # behind camera - put offscreen; return None to skip rendering if desired
        return None
    # focal length from fov (in degrees)
    f = (0.5 * SW) / math.tan(math.radians(fov) / 2.0)
    sx = SW/2 + (cx * f) / cz
    sy = SH/2 - (cy * f) / cz
    depth = cz
    return (int(sx), int(sy), depth)

# ---------------- Scene & rendering ----------------
def run_simulation(num_balls=120, method='BVH', world=300.0, radius=6.0):
    if not PYGAME_AVAILABLE:
        print("pygame nie jest zainstalowany. Zainstaluj pygame by uruchomić symulację.")
        return

    pygame.init()
    SW, SH = 1200, 800
    screen = pygame.display.set_mode((SW, SH))
    pygame.display.set_caption("3D Collision Detection - (b=Brute, s=SAP, v=BVH, SPACE pause)")
    clock = pygame.time.Clock()
    font = pygame.font.SysFont("Consolas", 16)

    N = num_balls

    # init balls
    balls = []
    for i in range(N):
        balls.append({
            'pos':[random.uniform(-world/2, world/2),
                   random.uniform(-world/2, world/2),
                   random.uniform(-world/2, world/2)],
            'vel':[random.uniform(-1.8,1.8),
                   random.uniform(-1.8,1.8),
                   random.uniform(-1.8,1.8)],
            'col':(0,200,0)
        })

    # initial aabbs & centers
    centers = [(b['pos'][0], b['pos'][1], b['pos'][2]) for b in balls]
    aabbs = [make_aabb_from_center_radius(c[0],c[1],c[2], radius) for c in centers]

    # build BVH once and get leaf_map for refit
    root, leaf_map = build_bvh_with_leaf_map(centers, aabbs)

    running = True
    paused = False
    show_method = method  # 'BRUTE', 'SAP', 'BVH'
    frame = 0

    # camera setup (orbiting around 0,0,0)
    cam = Camera(yaw=math.radians(45.0), pitch=math.radians(-20.0), distance=700.0, target=(0.0,0.0,0.0))
    rotating = False
    last_mouse = (0,0)
    pan_speed = 1.0

    # simple directional "light" in world space
    light_dir = (0.4, 0.7, 0.5)
    l_len = math.sqrt(sum(c*c for c in light_dir))
    light_dir = tuple(c / l_len for c in light_dir)

    while running:
        for ev in pygame.event.get():
            if ev.type == pygame.QUIT:
                running = False
            elif ev.type == pygame.KEYDOWN:
                if ev.key == pygame.K_ESCAPE:
                    running = False
                elif ev.key == pygame.K_SPACE:
                    paused = not paused
                elif ev.key == pygame.K_b:
                    show_method = 'BRUTE'
                elif ev.key == pygame.K_s:
                    show_method = 'SAP'
                elif ev.key == pygame.K_v:
                    show_method = 'BVH'
                elif ev.key == pygame.K_UP:
                    cam.target[1] += pan_speed * (cam.distance / 200.0)
                elif ev.key == pygame.K_DOWN:
                    cam.target[1] -= pan_speed * (cam.distance / 200.0)
                elif ev.key == pygame.K_LEFT:
                    cam.target[0] -= pan_speed * (cam.distance / 200.0)
                elif ev.key == pygame.K_RIGHT:
                    cam.target[0] += pan_speed * (cam.distance / 200.0)

            elif ev.type == pygame.MOUSEBUTTONDOWN:
                if ev.button == 1:
                    rotating = True
                    last_mouse = ev.pos
                elif ev.button == 4:  # wheel up -> zoom in
                    cam.distance = max(cam.min_distance, cam.distance * 0.9)
                elif ev.button == 5:  # wheel down -> zoom out
                    cam.distance = min(cam.max_distance, cam.distance * 1.1)

            elif ev.type == pygame.MOUSEBUTTONUP:
                if ev.button == 1:
                    rotating = False

            elif ev.type == pygame.MOUSEMOTION:
                if rotating:
                    mx, my = ev.pos
                    lx, ly = last_mouse
                    dx = mx - lx
                    dy = my - ly
                    # adjust yaw/pitch with sensitivity scaled by distance
                    sens = 0.005 * (cam.distance / 300.0)
                    cam.yaw += dx * sens
                    cam.pitch += dy * sens
                    # clamp pitch to avoid flip
                    cam.pitch = max(-math.pi/2 + 0.01, min(math.pi/2 - 0.01, cam.pitch))
                    last_mouse = ev.pos

        if not paused:
            # integrate simple motion + bounce inside world AABB
            for b in balls:
                for k in range(3):
                    b['pos'][k] += b['vel'][k]
                    if b['pos'][k] < -world/2 + radius:
                        b['pos'][k] = -world/2 + radius; b['vel'][k] *= -1
                    if b['pos'][k] > world/2 - radius:
                        b['pos'][k] = world/2 - radius; b['vel'][k] *= -1
                b['col'] = (0,200,0)

            # update centers & aabbs
            centers = [(b['pos'][0], b['pos'][1], b['pos'][2]) for b in balls]
            aabbs = [make_aabb_from_center_radius(c[0],c[1],c[2], radius) for c in centers]

            # collision detection based on chosen method
            if show_method == 'BVH':
                if root is None or not leaf_map:
                    root, leaf_map = build_bvh_with_leaf_map(centers, aabbs)
                else:
                    # update leaf nodes aabb
                    for i, a in enumerate(aabbs):
                        leaf = leaf_map.get(i)
                        if leaf is not None:
                            leaf.aabb = a
                        else:
                            root, leaf_map = build_bvh_with_leaf_map(centers, aabbs)
                            break
                    # refit tree bottom-up
                    refit_bvh(root)
                pairs = bvh_collisions_using_root(aabbs, root)
            elif show_method == 'SAP':
                pairs = sweep_and_prune_collisions(aabbs)
            else:
                pairs = brute_force_collisions(aabbs)

            # mark collisions (visual only)
            for i,j in pairs:
                balls[i]['col'] = (255,80,0)
                balls[j]['col'] = (255,80,0)

        # render
        screen.fill((12,12,16))

        # draw axis / bounding box (wireframe) - sample 12 edges
        # compute 8 corners of world box
        hw = world/2.0
        corners = [
            (-hw, -hw, -hw), ( hw, -hw, -hw), ( hw,  hw, -hw), (-hw,  hw, -hw),
            (-hw, -hw,  hw), ( hw, -hw,  hw), ( hw,  hw,  hw), (-hw,  hw,  hw),
        ]
        edges = [(0,1),(1,2),(2,3),(3,0),(4,5),(5,6),(6,7),(7,4),(0,4),(1,5),(2,6),(3,7)]
        edge_color = (60,60,60)
        for a,b in edges:
            pa = project_point(corners[a], cam, SW, SH)
            pb = project_point(corners[b], cam, SW, SH)
            if pa is not None and pb is not None:
                pygame.draw.aaline(screen, edge_color, (pa[0],pa[1]), (pb[0],pb[1]), 1)

        # draw balls: project, compute size from depth, sort by depth (far->near)
        drawlist = []
        for idx, b in enumerate(balls):
            proj = project_point(b['pos'], cam, SW, SH)
            if proj is None:
                continue
            sx, sy, depth = proj
            # simple size falloff: objects nearer appear bigger
            size = max(1, int((radius * (SW/2) / (depth + 1.0)) * 0.6))
            # shading: compute approximate normal dot light using vector from ball center to camera
            cam_pos = cam.get_position()
            vx = cam_pos[0] - b['pos'][0]
            vy = cam_pos[1] - b['pos'][1]
            vz = cam_pos[2] - b['pos'][2]
            vlen = math.sqrt(vx*vx + vy*vy + vz*vz) or 1e-6
            nx, ny, nz = vx/vlen, vy/vlen, vz/vlen  # approximate normal towards camera
            # dot with light
            ld = max(0.0, nx*light_dir[0] + ny*light_dir[1] + nz*light_dir[2])
            drawlist.append((depth, sx, sy, size, b['col'], ld))

        # sort far to near (large depth first) so nearer render last (on top)
        drawlist.sort(reverse=True, key=lambda e: e[0])
        for _, sx, sy, size, col, light_factor in drawlist:
            # compute shaded color
            r = min(255, int(col[0] * (0.3 + 0.7*light_factor)))
            g = min(255, int(col[1] * (0.3 + 0.7*light_factor)))
            bcol = min(255, int(col[2] * (0.3 + 0.7*light_factor)))
            # shadow offset for depth illusion
            pygame.draw.circle(screen, (10,10,10), (sx+int(size*0.2), sy+int(size*0.2)), max(1, int(size*0.9)))
            pygame.draw.circle(screen, (r,g,bcol), (sx, sy), size)

        info = f"Method: {show_method}  (b=Brute, s=SAP, v=BVH, SPACE pause)  Balls: {N}  World: {world}  Radius: {radius:.1f}"
        text = font.render(info, True, (220,220,220))
        screen.blit(text, (10,10))

        # camera debug
        cam_pos = cam.get_position()
        cam_txt = f"Cam dist: {cam.distance:.1f}  yaw:{math.degrees(cam.yaw):.1f}  pitch:{math.degrees(cam.pitch):.1f}"
        text2 = font.render(cam_txt, True, (180,180,180))
        screen.blit(text2, (10, 32))

        pygame.display.flip()
        clock.tick(60)
        frame += 1

        # occasional BVH rebuild (keeps tree balanced if many insertions / reorder)
        if show_method == 'BVH' and frame % 100 == 0:
            root, leaf_map = build_bvh_with_leaf_map(centers, aabbs)

    pygame.quit()

if __name__ == "__main__":
    # --- łatwe miejsca do zmiany wymiarów sceny i kul:
    # world: wielkość sześciennego 'boxa' (krawędź = world)
    # radius: promień kulek
    # num_balls: ilość kulek
    #
    # przykłady:
    # run_simulation(num_balls=80, method='BVH', world=600.0, radius=12.0)
    run_simulation(num_balls=700, method='SAP', world=600.0, radius=10.0)
